In [1]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
import keras
import tensorflow as tf
import numpy as np
from numpy  import expand_dims
import cv2
from matplotlib import pyplot as plt
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
np.random.seed(15)
import matplotlib.pyplot as plt
import PIL

Using TensorFlow backend.


# 개 고양이 인식

- https://keraskorea.github.io/posts/2018-10-24-little_data_powerful_model/
- https://www.kaggle.com/c/dogs-vs-cats/data
- 학습 데이터로 1,000장의 고양이 사진과 1,000장의 강아지 사진을 사용 (kaggle  25,000자)
- 검증 데이터로는 각각 400장 사용

## 1. Augmentation 사용 했을 때

In [2]:
#데이터 호출

batch_size = 16

train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

# 검증 및 테스트 이미지는 augmentation을 적용하지 않음(이미지 원본을 사용)
validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# 이미지를 배치 단위로 불러와 줄 generator입니다.
train_generator = train_datagen.flow_from_directory(
        'smallcatdog/train', 
        target_size=(150, 150), 
        batch_size=batch_size,
        class_mode='binary') 

validation_generator = validation_datagen.flow_from_directory(
        'smallcatdog/validation',
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='binary')

test_generator = test_datagen.flow_from_directory(
        'smallcatdog/validation',
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='binary')

Found 2000 images belonging to 2 classes.
Found 800 images belonging to 2 classes.
Found 800 images belonging to 2 classes.


In [3]:
#CNN 구현 


model = Sequential()


model.add(Conv2D(32, (3, 3), input_shape=(150, 150,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2))) 
#맥스풀링 여러개쓰는 이유는 작은사이즈의 고양이도 있기 때문에 작게 들어가서 잡아줄 필요가 있다.계층구조를 많이 주면 성능에 도움이 된다.


model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors


model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))


model.add(Dense(1))
model.add(Activation('sigmoid'))# 0~ 1값나옴

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [11]:
# steps_per_epoch는 한 세대마다 몇 번 생성기로부터 데이터를 얻을지를 나타내는 값
# 한 세대마다 사용되는 학습데이터의 수는 steps_per_epoch * batch_size
        
model.fit_generator(
        train_generator,
        steps_per_epoch=2000 // batch_size,    # 2000/16     한번에 125개씩 생성
        epochs=5,  #50
        validation_data=validation_generator,
        validation_steps=800 // batch_size)     # 800/16   한번에 50개씩 생성

#모델 저장
model.save("smallcatdog_use_aug.h5")
scores = model.evaluate_generator( test_generator,       steps = 5)
print(scores[1])

Epoch 1/5
125/125 [==============================] - 18s 141ms/step - loss: 0.2517 - accuracy: 0.9115 - val_loss: 1.3727 - val_accuracy: 0.7200
Epoch 2/5
125/125 [==============================] - 18s 143ms/step - loss: 0.2397 - accuracy: 0.9220 - val_loss: 0.4486 - val_accuracy: 0.7125
Epoch 3/5
125/125 [==============================] - 18s 141ms/step - loss: 0.2571 - accuracy: 0.9130 - val_loss: 2.5709 - val_accuracy: 0.7275
Epoch 4/5
125/125 [==============================] - 17s 139ms/step - loss: 0.2438 - accuracy: 0.9220 - val_loss: 1.4916 - val_accuracy: 0.7300
Epoch 5/5
125/125 [==============================] - 17s 138ms/step - loss: 0.2745 - accuracy: 0.9205 - val_loss: 1.8981 - val_accuracy: 0.7225
0.7875000238418579


In [12]:
# 모델 평가하기
print("-- Evaluate --")
scores = model.evaluate_generator( test_generator,       steps = 800/16) #전체 800개 , 한번에 16개 씩 가져옴
print(scores[1])

-- Evaluate --
0.7225000262260437


## 2. Augmentation 없이 학습
- 기본 이미지만 사용 했을 때
- scale 만 사용

In [7]:
# augmentation 없이  학습
batch_size = 16

train_datagen = ImageDataGenerator(rescale=1./255 )
validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# 이미지를 배치 단위로 불러와 줄 generator입니다.
train_generator = train_datagen.flow_from_directory(
        'smallcatdog/train',  # this is the target directory
        target_size=(150, 150),  # 모든 이미지의 크기가 150x150로 조정됩니다.
        batch_size=batch_size,
        class_mode='binary')  # binary_crossentropy 손실 함수를 사용하므로 binary 형태로 라벨을 불러와야 합니다.

validation_generator = validation_datagen.flow_from_directory(
        'smallcatdog/validation',
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='binary')

test_generator = test_datagen.flow_from_directory(
        'smallcatdog/validation',
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='binary')

Found 2000 images belonging to 2 classes.
Found 800 images belonging to 2 classes.
Found 800 images belonging to 2 classes.


In [9]:
#네트워크 생성
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(150, 150,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

model.fit_generator(
        train_generator,
        steps_per_epoch=2000 // batch_size,
        epochs=50, # 50
        validation_data=validation_generator,
        validation_steps=800 // batch_size)

Epoch 1/50
125/125 [==============================] - 18s 140ms/step - loss: 0.7383 - accuracy: 0.5220 - val_loss: 0.6874 - val_accuracy: 0.5263
Epoch 2/50
125/125 [==============================] - 18s 142ms/step - loss: 0.6698 - accuracy: 0.5925 - val_loss: 0.6642 - val_accuracy: 0.6550
Epoch 3/50
125/125 [==============================] - 18s 144ms/step - loss: 0.6130 - accuracy: 0.6695 - val_loss: 0.8499 - val_accuracy: 0.6900
Epoch 4/50
125/125 [==============================] - 18s 143ms/step - loss: 0.5769 - accuracy: 0.7000 - val_loss: 0.6919 - val_accuracy: 0.6825
Epoch 5/50
125/125 [==============================] - 17s 137ms/step - loss: 0.5442 - accuracy: 0.7275 - val_loss: 0.4901 - val_accuracy: 0.6875
Epoch 6/50
125/125 [==============================] - 18s 141ms/step - loss: 0.4808 - accuracy: 0.7765 - val_loss: 0.5820 - val_accuracy: 0.7250
Epoch 7/50
125/125 [==============================] - 18s 140ms/step - loss: 0.4602 - accuracy: 0.7870 - val_loss: 0.5028 - val_ac

In [10]:
#모델 저장
model.save("smallcatdog_without_aug.h5")
scores = model.evaluate_generator( test_generator,       steps = 5)
print(scores[1])

0.7250000238418579


## 3. Model 불러와서 재적용

In [2]:
from keras.models import load_model
model = load_model("smallcatdog_use_aug.h5")
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 148, 148, 32)      896       
_________________________________________________________________
activation_11 (Activation)   (None, 148, 148, 32)      0         
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 72, 72, 32)        9248      
_________________________________________________________________
activation_12 (Activation)   (None, 72, 72, 32)        0         
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 34, 34, 64)       

In [4]:
# augmentation 없이  학습
batch_size = 16

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
        'smallcatdog/validation',
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='binary')

scores = model.evaluate_generator( test_generator, steps = 800//16) #전체테스트사이즈/배치사이즈
print(scores[1])

Found 800 images belonging to 2 classes.
0.7225000262260437
